# 2 - Easiest Wins
##### **Author: Adam Gatt**

Later notebooks will cover more advance topics but the next two will aim to show off simple new language features that you should be able to immediately benefit from with little/no trade-off. Many revolve around enforcing correctness, to help remove some classes of bugs before they have a chance to happen. Additionally, they are largely compile-time features that leave no changes on the compiled code, resulting in no run-time cost. Notebook 4 will then cover a few topics that also bring coding improvements but may require a little thought to ensure you're doing what you want.

The [clang-tidy](https://clang.llvm.org/extra/clang-tidy/) linter will be able to make suggestions about many of these language features, pointing out situation where they can be used. With some care and manual review, you can even run the linter over an entire existing codebase and have the suggestions applied automatically.

## override

How come the duck below fails to realise it can quack?

In [1]:
#include <iostream>

class Bird {
    public:
    virtual void makeNoise() const {
        std::cout << "Chirp!" << std::endl;
    }
};

class Duck : public Bird {
    public:
    virtual void makeNoise() {
        std:: cout << "Quack!" << std::endl;
    }
};

Bird* daffy = new Duck();

daffy->makeNoise();

Chirp!


It's because the two `makeNoise()` functions aren't the same! Because one is const and the other isn't, they count as having different signatures. This means the derived class isn't overriding the makeNoise in the base class; the two methods don't occupy the same entry in the vtable. 

Constness is a subtle way this bug can occur, but it can also be caused by a typo in the method name or differences in parameter types.

Because both the base and overridden methods use the same syntax (beginning `virtual` annotation), the compiler doesn't have a clear way to know that `Duck::makeNoise` is meant to be an overridden method instead of starting its own base virtual method. `override` is a new annotation to add to the overriding method and make it clear to the compiler. When a method is marked `override` the compiler will search for a matching base virtual method to ensure it exists, and will throw a compilation error if one cannot be found.

As an added benefit it also makes the intention clear to the human reader.

In [2]:
class Turkey : public Bird {
    public:
    virtual void makeNoise() override {
        std:: cout << "Gobble!" << std::endl;
    }
};

input_line_9:3:18: error: 'makeNoise' marked 'override' but does not override any member functions
    virtual void makeNoise() override {
                 ^


Interpreter Error: 

## nullptr

`nullptr` is a Modern C++ keywork for the null reference, where in the past you might have used `0` or `NULL` (often #defined to be 0). The advantage of nullptr is that it has its own type (`std::nullptr_t`), whereas the `int` type of NULL can allow for subtle bugs or ambiguities, especially in constructors and overloaded functions. In the best-case scenario the compiler will notice the ambiguity with a warning or error, in the worst-case you may have a subtle logic error.

In [3]:
class Colour {    
    public:
    int rgbCode;
    
    // Specify the rgb code directly
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    
    // Copy the rgb code from another Colour
    Colour(const Colour* cloneSrc) {
        if (cloneSrc != NULL) {
            rgbCode = cloneSrc->rgbCode;
        }
    }
}

In [4]:
// Which constructor will be called?

Colour col1(NULL);

input_line_11:3:8: error: call to constructor of '__cling_N54::Colour' is ambiguous
Colour col1(NULL);
       ^    ~~~~
input_line_10:6:5: note: candidate constructor
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    ^
input_line_10:9:5: note: candidate constructor
    Colour(const Colour* cloneSrc) {
    ^
input_line_10:1:7: note: candidate is the implicit copy constructor
class Colour {    
      ^
input_line_10:1:7: note: candidate is the implicit move constructor


Interpreter Error: 

In [5]:
Colour col2(nullptr);

Notice the constructor executing correctly instead of trying to dereference nullptr. This is because the guard clause `if (cloneSrc != NULL)` still performed its job. The new keyword enjoys strong interoperability with existing legacy code as the rules for comparing nullptr and NULL have been well-defined to produce intuitive results.

In [1]:
bool comparisons[] {
    nullptr == nullptr,
    nullptr != nullptr,
    nullptr == NULL,
    nullptr != NULL,
    nullptr == 0,
    nullptr != 0,
    nullptr ? true : false
};

comparisons

13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr == NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:6:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr != NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:9:5: warning: implicit conversion of nullptr constant to 'bool' [-Wnull-conversion]
    nullptr ? true : false
    ^~~~~~~ ~
    false


{ true, false, true, false, true, false, false }

Another example: when creating a nullable reference to a _Line_ object we accidentally leave out the asterisk and get unexpected results that will throw no errors. We would think that `NULL`, an int, shouldn't be assignable to a Line object. But despite the `=` character in the bottom-most line, the operation that actually takes place here is  initialisation rather than assignment. As such the constructor is called with the argument `NULL`, which the compiler happily treats as the int that it is.

In [2]:
#include <iostream>

class Line {
    private:
    int thickness;
    
    public:
    Line(int thickness) : thickness(thickness) {
        std::cout << "Created line with thickness " << thickness;
    }
};

Line myLine = NULL;

Created line with thickness 0

Wait, _what_? Who even asked for a constructor to be called here? More on this in the next notebook with `explicit`.

## constexpr
`constexpr` is a new keyword for setting constant values that won't change during program execution. More specifically, constexpr represents values that are known (or can be calculated) at _compile time_. Its most promising feature is to provide in-language syntax to replace and improve upon preprocessor definitions, as well as some advantages over regular `const`.


In [3]:
constexpr int COLOUR_DEPTH{256}; // I like UPPER_SNAKE_CASE for constants
std::array<int, COLOUR_DEPTH> fixedBuffer; // Can use in places that require constant values

In [4]:
constexpr int MAX_BUFFER_SIZE{50};
std::vector<double> buffer;

In [5]:
void addToBuffer(double newValue) {
    buffer.push_back(newValue);
    if (buffer.size() > MAX_BUFFER_SIZE) {
        buffer.erase(buffer.begin());
    }
}

### Comparison to preprocessor definitions
Like all things preprocessor, a preprocessor definition operates on a textual level, interacting with the underlying source code with no consideration of its parseable syntax. `constexpr` however is firmly a part of the C++ language semantics, and thus brings these benefits:
* constexpr respects the use of type checking, templates, etc, and thus allows for much greater safety for implementing the concept of a compile-time value
* constexpr variables are debuggable whereas heavy use of preprocessor can disrupt the debugger
* constexpr variables respect scopes and namespaces whereas preprocessor definitions can pollute everything that comes after them if not #undef'd (looking at you [_windows.h_ defining _min()_ and _max()_](https://belaycpp.com/2021/05/11/windows-h-breaks-the-stl-and-my-will-to-live/))
* Finally, constexpr expressions follow a syntax that simply reads more like standard C++ code and thus requires less context-switching when reading and is easier to deal with when on-boarding newer devs.

In [6]:
// What types do these variables have?
// (Probably a trick question, as none of them will actually be variables at all)
#define DEFAULT_EXPERIENCE 10
#define PLAYERS 4
#define DEFAULT_EXPERIENCE_PER_PLAYER DEFAULT_EXPERIENCE / PLAYERS

DEFAULT_EXPERIENCE_PER_PLAYER

2

### Comparison to const variables
Despite the similarity in name, constexpr variables go significantly further than const:
 * const variables cannot be changed once defined, but this initial definition occurs at _run-time_ and can potentially be a good while into program execution. 
 * constexpr variables **must** be completely known at _compile-time_.

In [8]:
#include <string>
#include <iostream>

const std::string name = [](){
    std::string input;
    std::cin >> input;
    return input;
}(); // The "Immediately invoked function expression" idiom

std::cout << "Hello, " << name << " :)";

Adam
Hello, Adam :)

### constexpr functions
You can add the `constexpr` annotation to a function to indicate to the compiler that the function is expected to produce a constexpr output. The only way that this output can be constexpr (i.e. computable at compile-time) is if all inputs and other data sources are known at compile-time.

In [1]:
// Ideally we define our input parameter as const, but let's not for a moment
constexpr double calcUpdatesPerDay(int updateRate_ms) {
    return (1000.0 / updateRate_ms) * 60 * 60 * 24; 
}

constexpr int UPDATE_RATE_ms = 500;

// Generation of constexpr output is successful as the compiler knows that UPDATE_RATE_ms is constexpr
constexpr double UPDATES_PER_DAY = calcUpdatesPerDay(UPDATE_RATE_ms);

UPDATES_PER_DAY

In [2]:
// Failed assigned from constexpr function to constexpr variable.
// The compiler knows that userUpdateRate_ms is not constexpr and so the function will
// not actually return a constexpr output

#include <iostream>

int userUpdateRate_ms{};

std::cin >> userUpdateRate_ms;

constexpr double NEW_UPDATES_PER_DAY = calcUpdatesPerDay(userUpdateRate_ms);

In [4]:
// The non-constexpr output of this function call can still be assigned to a non-constexpr variable

#include <iostream>

int userUpdateRate_ms;

std::cin >> userUpdateRate_ms;

double newUpdatesPerDay = calcUpdatesPerDay(userUpdateRate_ms);
newUpdatesPerDay

20


4320000.0

This behaviour makes the `constexpr` annotation for functions act like suggestion to the compiler (like `inline`).
 * If we expect the function to provide outputs that are known at compile-time then we should use constexpr.
 * This allows for great compiler optimisation. For example the function itself can actually be run during compilation and thus its actual bytecode can be completely excluded from the program binary.
 * But the function will not be limited to that behaviour. If it is used in any non-constexpr context it becomes callable like any regular function. 

## using
The `using` keyword is an improved alternative to `typedef`, used for creating aliases based on existing types. `using` comes with a cleaner syntax with better readability, and offers new functionality in that it has support for templates.

In [1]:
constexpr int width = 100;
constexpr int height = 150;

// New type alias syntax
using Image = std::array<std::array<unsigned int, width>, height>;

// Avoid repeating the lengthy custom type
Image backbuffer;
Image canvas;
Image depthMap;


// Typedef syntax for comparison
typedef std::array<std::array<unsigned int, width>, height> Image2;

In [2]:
// The type alias avoids a lengthy parameter type
unsigned int pixelAt(Image image, int column, int row) {
        return image[row][column];
}

### Type alias to function signature
Whereas the `using` syntax appears somewhat cleaner for standard type declarations, it is **much** cleaner when used for types based on function signatures. This can be seen in the example below, where the typedef-style declaration requires the name to be buried in the middle of the expression, between the parameters and return type.

In [4]:
#include <string>

// We want a type for a function that accepts two strings and returns an int

// Using syntax
using StrComp = int (*) (std::string, std::string);

// Typedef syntax
typedef int (*StrComp2) (std::string, std::string);

In [7]:
std::string getFirstString(std::string string1, std::string string2, StrComp comparator) {
    return (comparator(string1, string2) < 0)
        ? string1
        : string2;
}

### Alias templates
But the biggest benefit to the new alias syntax is the new functionality of supporting `template`s, which typedefs cannot do. This allows `using`-based definitions to becomes partially-specialised, accepting an additional parameter to fully define the type.

In [9]:
#include <utility>

using Dimensions = std::pair<int, int>;

In [10]:
#include <array>

template <size_t length>
using ResultsList = std::array<Dimensions, length>;

ResultsList<500> fullResults;
ResultsList<10> paginatedResults;

In [11]:
// Or, if we want, we can complete the specialisation with an additional type that provides the template value
using PaginatedResultsList = ResultsList<10>;